In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('data.csv', low_memory=False)

In [3]:
#dropping irrelevant games and junk column
df = df[df.GameMode != 'ARAM']
df = df.drop(columns='Unnamed: 0')

#creating a teamID to signify the player's team
df['win'] = df['win'].astype(int)
df['teamID'] = df['GameID'] + df['win'].astype(str)

In [4]:
df

,assists,GameID,GameMode,baronKills,bountyLevel,champExperience,champLevel,championName,damageDealtToObjectives,damageSelfMitigated,...,trueDamageDealtToChampions,trueDamageTaken,turretTakedowns,turretsLost,visionScore,visionWardsBoughtInGame,wardsKilled,wardsPlaced,win,teamID
1,3,4035079604,CLASSIC,0,3,15453,16,Sett,18147,30659,...,2556,686,4,4,10,0,0,7,1,40350796041
2,19,4035079604,CLASSIC,0,3,13168,15,Trundle,27626,28297,...,956,1571,6,4,21,3,0,11,1,40350796041
3,6,4035079604,CLASSIC,0,1,13619,15,Sylas,10629,19937,...,600,935,6,4,8,0,0,5,1,40350796041
4,7,4035079604,CLASSIC,1,1,10674,13,KogMaw,13624,6303,...,2202,521,3,4,3,1,1,2,1,40350796041
5,24,4035079604,CLASSIC,0,0,11295,13,Nami,2707,5087,...,1284,352,5,4,68,5,7,31,1,40350796041
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99996,5,4035205190,CLASSIC,0,0,10796,13,Mordekaiser,1137,15053,...,528,1134,0,10,16,1,0,5,0,40352051900
99997,2,4035205190,CLASSIC,0,0,10318,13,MasterYi,6707,11223,...,5998,1147,0,10,14,1,0,7,0,40352051900
99998,4,4035205190,CLASSIC,0,0,9898,12,Qiyana,337,8258,...,1077,433,0,10,16,1,0,9,0,40352051900
99999,1,4035205190,CLASSIC,0,0,8304,11,Kalista,3931,11403,...,272,116,0,10,17,1,3,9,0,40352051900


In [5]:
#df of teams and win or loss
df_teams = df.groupby(by=['teamID']).agg({'championName' : lambda x: ' '.join(x), 'win' : 'sum'})
df_teams.rename(columns = {'championName':'champs'}, inplace = True)

df_teams

,champs,win
teamID,,
40350796040,Yone Viego Yasuo Caitlyn Swain,0
40350796041,Sett Trundle Sylas KogMaw Nami,5
40350796130,Garen FiddleSticks Fizz Caitlyn Swain,0
40350796131,Malzahar Nocturne Riven Sivir Lux,5
40350796650,TahmKench Kayn Veigar Caitlyn Blitzcrank,0
...,...,...
40352051671,Camille Warwick Leblanc Samira Sona,5
40352051780,Nasus Warwick Katarina Samira Rakan,0
40352051781,Gangplank MasterYi Akshan Twitch Teemo,5


In [6]:
#splitting each character into its own column
df_teams[['champ1','champ2','champ3','champ4','champ5']] = df_teams.champs.str.split(expand=True)
df_teams = df_teams.drop(columns='champs')
df_teams



,win,champ1,champ2,champ3,champ4,champ5
teamID,,,,,,
40350796040,0,Yone,Viego,Yasuo,Caitlyn,Swain
40350796041,5,Sett,Trundle,Sylas,KogMaw,Nami
40350796130,0,Garen,FiddleSticks,Fizz,Caitlyn,Swain
40350796131,5,Malzahar,Nocturne,Riven,Sivir,Lux
40350796650,0,TahmKench,Kayn,Veigar,Caitlyn,Blitzcrank
...,...,...,...,...,...,...
40352051671,5,Camille,Warwick,Leblanc,Samira,Sona
40352051780,0,Nasus,Warwick,Katarina,Samira,Rakan
40352051781,5,Gangplank,MasterYi,Akshan,Twitch,Teemo


In [7]:
#get support for each lane for calculations
#using 100 as our support threshold 
support_top = df_teams['champ1'].value_counts()
df_supp_top = pd.DataFrame(support_top)

support_jun = df_teams['champ2'].value_counts()
df_supp_jun = pd.DataFrame(support_jun)

support_mid = df_teams['champ3'].value_counts()
df_supp_mid = pd.DataFrame(support_mid)

support_bot = df_teams['champ4'].value_counts()
df_supp_bot = pd.DataFrame(support_bot)

support_sup = df_teams['champ5'].value_counts()
df_supp_sup = pd.DataFrame(support_sup)
#df_supp_sup = df_supp_sup[df_supp_sup['champ5'] > 100] 

In [8]:
df_support = pd.concat([df_supp_top, df_supp_jun, df_supp_mid, df_supp_bot, df_supp_sup])
df_support

,champ1,champ2,champ3,champ4,champ5
Sett,1052.0,NaN,NaN,NaN,NaN
Garen,884.0,NaN,NaN,NaN,NaN
Teemo,849.0,NaN,NaN,NaN,NaN
Nasus,796.0,NaN,NaN,NaN,NaN
Mordekaiser,775.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...
Kassadin,NaN,NaN,NaN,NaN,1.0
Yorick,NaN,NaN,NaN,NaN,1.0
Quinn,NaN,NaN,NaN,NaN,1.0
Ryze,NaN,NaN,NaN,NaN,1.0


In [9]:
#creating a df with just champ names and support values
df_support = df_support.reset_index()
df_support.rename(columns = {'index':'champ'}, inplace = True)
df_support = df_support.fillna(value='0')
df_support['count'] = df_support['champ1'].astype(float) + df_support['champ2'].astype(float) + df_support['champ3'].astype(float) + df_support['champ4'].astype(float) + df_support['champ5'].astype(float)
df_support = df_support.drop(columns=['champ1','champ2','champ3','champ4','champ5'])
df_support = df_support.groupby(by='champ').sum()

#dividing by number of teams (baskets) in our dataset
df_support['support'] = df_support['count'] / 20000
df_support = df_support.reset_index()
df_support


,champ,count,support
0,Aatrox,418.0,0.02090
1,Ahri,691.0,0.03455
2,Akali,589.0,0.02945
3,Akshan,1011.0,0.05055
4,Alistar,378.0,0.01890
...,...,...,...
151,Zed,939.0,0.04695
152,Ziggs,744.0,0.03720
153,Zilean,388.0,0.01940
154,Zoe,232.0,0.01160


In [10]:
#finding the number of frequent items
frequent = df_support['support'][df_support['support'] > 0.08].count()
frequent

#narrows us down to 14 frequent items

5

In [11]:
#support threshold of 6% applied
df_frequent = df_support
df_frequent['support'] = df_frequent['support'].where(df_frequent['support'] > 0.08, other = 0)
df_frequent

,champ,count,support
0,Aatrox,418.0,0.0
1,Ahri,691.0,0.0
2,Akali,589.0,0.0
3,Akshan,1011.0,0.0
4,Alistar,378.0,0.0
...,...,...,...
151,Zed,939.0,0.0
152,Ziggs,744.0,0.0
153,Zilean,388.0,0.0
154,Zoe,232.0,0.0


In [12]:
#just our frequent champs
frequent_champs = df_frequent
frequent_champs = frequent_champs[frequent_champs.support != 0]
frequent_champs


,champ,count,support
9,Ashe,2023.0,0.10115
16,Caitlyn,1747.0,0.08735
28,Ezreal,1685.0,0.08425
48,Jhin,1881.0,0.09405
70,Lux,1977.0,0.09885


In [13]:
#now we must find the frequent pairs within each basket (team)
df_teams

,win,champ1,champ2,champ3,champ4,champ5
teamID,,,,,,
40350796040,0,Yone,Viego,Yasuo,Caitlyn,Swain
40350796041,5,Sett,Trundle,Sylas,KogMaw,Nami
40350796130,0,Garen,FiddleSticks,Fizz,Caitlyn,Swain
40350796131,5,Malzahar,Nocturne,Riven,Sivir,Lux
40350796650,0,TahmKench,Kayn,Veigar,Caitlyn,Blitzcrank
...,...,...,...,...,...,...
40352051671,5,Camille,Warwick,Leblanc,Samira,Sona
40352051780,0,Nasus,Warwick,Katarina,Samira,Rakan
40352051781,5,Gangplank,MasterYi,Akshan,Twitch,Teemo


In [14]:
#iterate through each champ row and check if its value is contained in the frequent champs table?
df_teams['c1isFrequent'] = df_teams['champ1'].isin(frequent_champs['champ']).astype(int)
df_teams['c2isFrequent'] = df_teams['champ2'].isin(frequent_champs['champ']).astype(int)
df_teams['c3isFrequent'] = df_teams['champ3'].isin(frequent_champs['champ']).astype(int)
df_teams['c4isFrequent'] = df_teams['champ4'].isin(frequent_champs['champ']).astype(int)
df_teams['c5isFrequent'] = df_teams['champ5'].isin(frequent_champs['champ']).astype(int)

df_teams

,win,champ1,champ2,champ3,champ4,champ5,c1isFrequent,c2isFrequent,c3isFrequent,c4isFrequent,c5isFrequent
teamID,,,,,,,,,,,
40350796040,0,Yone,Viego,Yasuo,Caitlyn,Swain,0,0,0,1,0
40350796041,5,Sett,Trundle,Sylas,KogMaw,Nami,0,0,0,0,0
40350796130,0,Garen,FiddleSticks,Fizz,Caitlyn,Swain,0,0,0,1,0
40350796131,5,Malzahar,Nocturne,Riven,Sivir,Lux,0,0,0,0,1
40350796650,0,TahmKench,Kayn,Veigar,Caitlyn,Blitzcrank,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
40352051671,5,Camille,Warwick,Leblanc,Samira,Sona,0,0,0,0,0
40352051780,0,Nasus,Warwick,Katarina,Samira,Rakan,0,0,0,0,0
40352051781,5,Gangplank,MasterYi,Akshan,Twitch,Teemo,0,0,0,0,0


In [15]:
#check if a row has at least two True values, otherwise drop it

df_teams['numFreq'] = df_teams['c1isFrequent'] + df_teams['c2isFrequent'] + df_teams['c3isFrequent'] + df_teams['c4isFrequent'] + df_teams['c5isFrequent']
df_teams = df_teams[df_teams['numFreq'] > 1]
df_teams

,win,champ1,champ2,champ3,champ4,champ5,c1isFrequent,c2isFrequent,c3isFrequent,c4isFrequent,c5isFrequent,numFreq
teamID,,,,,,,,,,,,
40350796651,5,Darius,Kindred,Vayne,Jhin,Lux,0,0,0,1,1,2
40350895791,5,Yorick,Amumu,Yone,Caitlyn,Lux,0,0,0,1,1,2
40350896241,5,Graves,Kindred,Fizz,Caitlyn,Lux,0,0,0,1,1,2
40350896321,5,Fiora,Ekko,Nunu,Caitlyn,Lux,0,0,0,1,1,2
40350896901,5,Garen,Nunu,Katarina,Jhin,Lux,0,0,0,1,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...
40352048410,0,Mordekaiser,Viego,Vladimir,Jhin,Lux,0,0,0,1,1,2
40352048741,5,Darius,Nocturne,Lux,Draven,Ashe,0,0,1,0,1,2
40352048890,0,Lucian,Olaf,Lux,Caitlyn,Velkoz,0,0,1,1,0,2


In [16]:
df_teams['champ1'] = df_teams['champ1'] + df_teams['c1isFrequent'].astype(str)
df_teams['champ2'] = df_teams['champ2'] + df_teams['c2isFrequent'].astype(str)
df_teams['champ3'] = df_teams['champ3'] + df_teams['c3isFrequent'].astype(str)
df_teams['champ4'] = df_teams['champ4'] + df_teams['c4isFrequent'].astype(str)
df_teams['champ5'] = df_teams['champ5'] + df_teams['c5isFrequent'].astype(str)

df_teams = df_teams.drop(columns=['c1isFrequent','c2isFrequent','c3isFrequent','c4isFrequent','c5isFrequent'])

df_teams

/var/folders/bt/g3_qyjjx3wx0n298n_kns6040000gn/T/ipykernel_44403/65264366.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_teams['champ1'] = df_teams['champ1'] + df_teams['c1isFrequent'].astype(str)
/var/folders/bt/g3_qyjjx3wx0n298n_kns6040000gn/T/ipykernel_44403/65264366.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_teams['champ2'] = df_teams['champ2'] + df_teams['c2isFrequent'].astype(str)
/var/folders/bt/g3_qyjjx3wx0n298n_kns6040000gn/T/ipykernel_44403/65264366.py:3: SettingWithCopyWarning

,win,champ1,champ2,champ3,champ4,champ5,numFreq
teamID,,,,,,,
40350796651,5,Darius0,Kindred0,Vayne0,Jhin1,Lux1,2
40350895791,5,Yorick0,Amumu0,Yone0,Caitlyn1,Lux1,2
40350896241,5,Graves0,Kindred0,Fizz0,Caitlyn1,Lux1,2
40350896321,5,Fiora0,Ekko0,Nunu0,Caitlyn1,Lux1,2
40350896901,5,Garen0,Nunu0,Katarina0,Jhin1,Lux1,2
...,...,...,...,...,...,...,...
40352048410,0,Mordekaiser0,Viego0,Vladimir0,Jhin1,Lux1,2
40352048741,5,Darius0,Nocturne0,Lux1,Draven0,Ashe1,2
40352048890,0,Lucian0,Olaf0,Lux1,Caitlyn1,Velkoz0,2


In [17]:
#get just the frequent champions
df_teams['champ1'] = np.where(df_teams["champ1"].str[-1]=="1",df_teams['champ1'],0)
df_teams['champ2'] = np.where(df_teams["champ2"].str[-1]=="1",df_teams['champ2'],0) 
df_teams['champ3'] = np.where(df_teams["champ3"].str[-1]=="1",df_teams['champ3'],0) 
df_teams['champ4'] = np.where(df_teams["champ4"].str[-1]=="1",df_teams['champ4'],0) 
df_teams['champ5'] = np.where(df_teams["champ5"].str[-1]=="1",df_teams['champ5'],0) 

df_teams['champ1'] = df_teams['champ1'].str[:-1]
df_teams['champ2'] = df_teams['champ2'].str[:-1]
df_teams['champ3'] = df_teams['champ3'].str[:-1] 
df_teams['champ4'] = df_teams['champ4'].str[:-1]
df_teams['champ5'] = df_teams['champ5'].str[:-1] 

df_teams


,win,champ1,champ2,champ3,champ4,champ5,numFreq
teamID,,,,,,,
40350796651,5,NaN,NaN,NaN,Jhin,Lux,2
40350895791,5,NaN,NaN,NaN,Caitlyn,Lux,2
40350896241,5,NaN,NaN,NaN,Caitlyn,Lux,2
40350896321,5,NaN,NaN,NaN,Caitlyn,Lux,2
40350896901,5,NaN,NaN,NaN,Jhin,Lux,2
...,...,...,...,...,...,...,...
40352048410,0,NaN,NaN,NaN,Jhin,Lux,2
40352048741,5,NaN,NaN,Lux,NaN,Ashe,2
40352048890,0,NaN,NaN,Lux,Caitlyn,NaN,2


In [18]:
#pull pairs out
df_teams = df_teams.fillna(value='')
df_teams['freqPairs'] = df_teams[['champ1', 'champ2', 'champ3', 'champ4', 'champ5']].agg(' '.join, axis=1)
df_teams

,win,champ1,champ2,champ3,champ4,champ5,numFreq,freqPairs
teamID,,,,,,,,
40350796651,5,,,,Jhin,Lux,2,Jhin Lux
40350895791,5,,,,Caitlyn,Lux,2,Caitlyn Lux
40350896241,5,,,,Caitlyn,Lux,2,Caitlyn Lux
40350896321,5,,,,Caitlyn,Lux,2,Caitlyn Lux
40350896901,5,,,,Jhin,Lux,2,Jhin Lux
...,...,...,...,...,...,...,...,...
40352048410,0,,,,Jhin,Lux,2,Jhin Lux
40352048741,5,,,Lux,,Ashe,2,Lux Ashe
40352048890,0,,,Lux,Caitlyn,,2,Lux Caitlyn


In [19]:
frequent_pairs = df_teams['freqPairs'].value_counts()
df_frequent_pairs = pd.DataFrame(frequent_pairs)
df_frequent_pairs = df_frequent_pairs.reset_index()
df_frequent_pairs.rename(columns = {'index':'pair', 'freqPairs':'count'}, inplace = True)
df_frequent_pairs['support'] = df_frequent_pairs['count'] / 20000
df_frequent_pairs

,pair,count,support
0,Jhin Lux,157,0.00785
1,Caitlyn Lux,154,0.00770
2,Ashe Lux,142,0.00710
3,Ezreal Lux,131,0.00655
4,Lux Caitlyn,61,0.00305
5,Lux Jhin,58,0.00290
6,Lux Ezreal,55,0.00275
7,Lux Ashe,48,0.00240
8,Ezreal Ashe,11,0.00055
9,Caitlyn Ashe,9,0.00045
